In [ ]:
!pip install glob2 peft wandb datasets trl==0.8.5 transformers accelerate -q
!pip install -U bitsandbytes flash_attn -q

In [ ]:
import tqdm
from llavadataset import llavadataset, collate_fn
import pickle
import peft
from peft import LoraConfig
from transformers import AutoTokenizer,BitsAndBytesConfig, AutoModelForCausalLM, CLIPVisionModel, AutoProcessor
import torch
from torch.utils.data import random_split, DataLoader
import pandas as pd
from torch.nn import functional as F
import csv

import random
from PIL import Image
import requests
import wandb
import os
from peft import PeftModel
import torch.nn as nn

import torch
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Colab_Notebooks/Session30

In [ ]:
# import wandb
# from google.colab import userdata
# wandb1 = userdata.get('wandb')
# os.environ["WANDB_API_KEY"] = wandb1
os.environ["WANDB_API_KEY"] = "ABABABABABABABA"

clip_model_name = "openai/clip-vit-base-patch32"
phi_model_name  = "microsoft/phi-2"
tokenizer  = AutoTokenizer.from_pretrained(phi_model_name, trust_remote_code=True)
processor  = AutoProcessor.from_pretrained(clip_model_name)
tokenizer.add_tokens('[QA]')
tokenizer.add_special_tokens({'pad_token':'[PAD]'})
train_batch_size = 64
clip_embed = 768
phi_embed  = 2560
device = "cuda" if torch.cuda.is_available() else "cpu"
num_workers = 10
IMAGE_TOKEN_ID = 23893 # token for word comment
max_steps      = 100000
EOS_TOKEN_ID   = 50256
#phi_patches    = 49
vocab_size     = 51200
max_generate_length = 100
model_val_step      = 100
model_log_step      = 100
model_save_step     = 100

wandb.init(project="clip_phi2_project", name="clip_phi3_finetune")
torch.set_float32_matmul_precision('medium')
tokenizer.pad_token_id, tokenizer.eos_token_id, tokenizer('[QA]')

/home/cudaq/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: shivdutta. Use `wandb login --relogin` to force relogin


(50296, 50256, {'input_ids': [50295], 'attention_mask': [1]})

In [ ]:
# training data
csv_file = 'train_token.csv'
qa_dataset = pd.read_csv(csv_file).head(200000)

# data loaders
train_dataloader = DataLoader(llavadataset(qa_dataset, phi_model_name,clip_model_name,processor),
                                           collate_fn=collate_fn, batch_size=train_batch_size, num_workers = num_workers, shuffle=True, pin_memory=True)

In [ ]:
len(train_dataloader)

3125

In [ ]:
# load validation data
file = open('sample_val_data.csv')
csvreader = csv.reader(file)
sample_val_data = []
for row in csvreader:
    sample_val_data.append(row)
print(sample_val_data[1])
file.close()

['http://images.cocodataset.org/train2017/000000033471.jpg', 'What are the colors of the bus in the image? [QA]', 'The bus in the image is white and red.<|endoftext|>']


In [ ]:
class SimpleResBlock(nn.Module):
    def __init__(self, phi_embed):
        super().__init__()
        # Layer Normalization: Normalizes the input to have zero mean and unit variance across the feature dimension.
        # It helps stabilize the training by ensuring that the values passed through the network remain in a consistent range.
        self.pre_norm = nn.LayerNorm(phi_embed)
        # nn.Sequential defines a sequential container, meaning a series of layers applied one after another.
        # nn.Linear(phi_embed, phi_embed): A fully connected (linear) layer that takes an input of size phi_embed and outputs the same size (phi_embed).
        # GELU (Gaussian Error Linear Unit) is an activation function that smooths out nonlinearities.
        # It's similar to ReLU, but it has a smoother gradient, which can improve performance in some architectures.
        self.proj = nn.Sequential(
            nn.Linear(phi_embed, phi_embed),
            nn.GELU(),
            nn.Linear(phi_embed, phi_embed)
        )
    # This method defines how the input x passes through the block during forward propagation.
    # It takes an input tensor x, normalizes it, processes it through two linear layers with a GELU activation in between, and then adds the original input x back to the output of the transformation.
    # The residual connection is important because it allows the model to retain information from earlier layers and makes it easier to train deep models.
    def forward(self, x):
        # key part of a residual block
        x = self.pre_norm(x)
        return x + self.proj(x)

In [ ]:
# This code is setting up two different models: a vision model (CLIPVisionModel)
# and a language model (AutoModelForCausalLM), along with a few additional components
# such as a projection layer and a residual block
clip_model = CLIPVisionModel.from_pretrained(clip_model_name).to(device)

# BitsAndBytesConfig: This is a configuration for quantizing the model weights, typically to reduce memory usage
# and speed up computations. It's part of Hugging Face's "bitsandbytes" integration for efficient model loading.
# load_in_4bit=True: Specifies that the model should be loaded in 4-bit precision.
# This significantly reduces the model's memory footprint compared to 16-bit or 32-bit precision, but at the cost of some precision.
# bnb_4bit_quant_type="nf4": Specifies the quantization type for 4-bit loading,
# in this case, "nf4" (Normal Float 4). NF4 is a specialized quantization format designed for stability during fine-tuning.
# bnb_4bit_compute_dtype=torch.bfloat16: Specifies that during computation, the model will use bfloat16 (a 16-bit floating-point format) for better performance while maintaining sufficient accuracy for deep learning tasks. bfloat16 is widely used in modern deep learning hardware.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16)

# _attn_implementation="flash_attention_2": This argument specifies the use of "Flash Attention 2", a more efficient attention mechanism that speeds up transformer
# models by reducing memory usage during attention computation. This improves training/inference speed, particularly in large models.
# torch_dtype="auto": This sets the model's data type automatically, depending on the available hardware and model precision requirements (e.g., floating-point 16 or 32-bit).
# quantization_config=bnb_config: The BitsAndBytesConfig from earlier is passed here to load the Phi model in 4-bit precision.
# trust_remote_code=True: This flag allows loading of custom code from the model repository, enabling the use of specialized code for the Phi model.
phi_model = AutoModelForCausalLM.from_pretrained(
    phi_model_name,
    _attn_implementation="flash_attention_2",
    torch_dtype="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)
phi_model.config.use_cache = False
# torch.nn.Linear(clip_embed, phi_embed): A linear projection layer that maps the image embeddings produced by the CLIP model
# (clip_embed size) to the dimensionality of the Phi model's embeddings (phi_embed size).
# clip_embed: The dimensionality of the CLIP image embeddings (e.g., 768 for CLIP ViT).
# phi_embed: The dimensionality of the Phi model's token embeddings (e.g., 2560).
projection = torch.nn.Linear(clip_embed, phi_embed).to(device)
# SimpleResBlock(phi_embed): A residual block that takes the projected image embeddings and processes them further.
# This helps the model better integrate image and text features.
resblock = SimpleResBlock(phi_embed).to(device)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# LoRA (Low-Rank Adaptation) configuration for the phi_model, a causal language model, using the PEFT (Parameter-Efficient Fine-Tuning) library
from peft import LoraConfig

# lora_alpha = 16: This is a scaling factor for the low-rank matrices in LoRA.
# It controls how much influence the low-rank adaptation has on the final output. A higher alpha means more scaling.
# lora_dropout = 0.1: This is the dropout probability applied to the LoRA layers.
# It adds regularization to avoid overfitting by randomly dropping out units during training.
# lora_r = 64: This is the rank of the low-rank matrices. LoRA approximates updates to large models using low-rank matrices to reduce the number of trainable parameters.
# The higher the value of r, the more trainable parameters there are.
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64


# task_type="CAUSAL_LM": Specifies that the task is for Causal Language Modeling, meaning the model generates tokens based on previous ones (autogressive task).
# This is typical for models like GPT, where each token depends on the previous tokens.
# target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj']:
# k_proj (key projection), q_proj (query projection), v_proj (value projection), and o_proj (output projection) refer to components in the multi-head self-attention layers.
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
)

# peft.get_peft_model(phi_model, peft_config): This function takes the pre-trained phi_model (the causal language model)
# and applies the LoRA configuration (peft_config). It returns a modified version of the phi_model,
# where only the selected layers (projections in this case) are fine-tuned using LoRA, while the rest of the model remains frozen.
peft_model = peft.get_peft_model(phi_model, peft_config).to(device)
peft_model.print_trainable_parameters()

trainable params: 31,457,280 || all params: 2,811,141,120 || trainable%: 1.1190


In [ ]:
# clip non trainable
# freezes the parameters of the clip_model, making it non-trainable during the fine-tuning process.
for network in [clip_model]:
    for param in network.parameters():
        param.requires_grad_(False)

In [ ]:
# check trainable paramaeters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"PEFT MODEL:{count_parameters(peft_model)}")
print(f"PROJECTION MODEL:{count_parameters(projection)}")
print(f"CLIP MODEL:{count_parameters(clip_model)}")
print(f"PHI MODEL:{count_parameters(phi_model)}")
print(f"RESNET MODEL:{count_parameters(resblock)}")

PEFT MODEL:31457280
PROJECTION MODEL:1968640
CLIP MODEL:0
PHI MODEL:31457280
RESNET MODEL:13117440


In [ ]:
print(peft_model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2560)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x PhiDecoderLayer(
            (self_attn): PhiFlashAttention2(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=2560, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): Module

In [ ]:
# load the model if present
projection.load_state_dict(torch.load('model_chkpt/clipphi_proj.pth'))
resblock.load_state_dict(torch.load('model_chkpt/clipphi_resblock.pth'))
print("Loaded step1 checkpoint")

Loaded step1 checkpoint


/tmp/ipykernel_45801/511553829.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  projection.load_state_dict(torch.load('model_chkpt/clipphi_proj.pth'))
/tmp/ipykernel_4580

In [ ]:
# function model_run_val that performs a random validation prediction using a pre-trained model
# random validation prediction
def model_run_val(sample_val_data,max_generate_length=10):

    # Randomly selects a datapoint from sample_val_data using random.randrange. The first entry (index 0) is assumed to be the header, so the range starts from 1.
    total_val_len = len(sample_val_data)
    random_val_datapoint = random.randrange(1,total_val_len) # 0 is header

    # Extracts the image URL, question (val_q), and answer (val_a) from the selected validation datapoint.
    val_image_url = sample_val_data[random_val_datapoint][0]
    val_q = sample_val_data[random_val_datapoint][1]
    val_a = sample_val_data[random_val_datapoint][2]

    with torch.no_grad():
        # Loads the image from the URL using requests and processes it using processor. The processed image tensor is moved to the GPU (or device).
        image_load = Image.open(requests.get(val_image_url,stream=True).raw)
        image_processed = processor(images=image_load, return_tensors="pt").to(device)

        # The image is passed through the CLIP model (clip_model) to obtain image embeddings.
        # These embeddings are processed using a projection layer and a residual block (resblock), and then converted to float16.
        clip_val_outputs = clip_model(**image_processed).last_hidden_state[:,1:,:]
        val_image_embeds = projection(clip_val_outputs)
        val_image_embeds = resblock(val_image_embeds).to(torch.float16)

        # The image token (represented by IMAGE_TOKEN_ID) is embedded using the PEFT model's embedding layer.
        img_token_tensor = torch.tensor(IMAGE_TOKEN_ID).to(device)
        img_token_embeds = peft_model.model.model.embed_tokens(img_token_tensor).unsqueeze(0).unsqueeze(0)

        # The question (val_q) is tokenized using the tokenizer, and the resulting tokens are embedded using the PEFT model's embedding layer.
        val_q_tokenised = tokenizer(val_q, return_tensors="pt", return_attention_mask=False)['input_ids'].squeeze(0)
        val_q_embeds  = peft_model.model.model.embed_tokens(val_q_tokenised).unsqueeze(0)

        # The image embeddings, image token embeddings, and question embeddings are concatenated to form a combined embedding input for the model.
        val_combined_embeds = torch.cat([val_image_embeds, img_token_embeds, val_q_embeds], dim=1) # 1, 69, 2560

        # A tensor is initialized to hold the predicted caption. The value 50256 is a special token, possibly the end-of-sequence (EOS) token.
        predicted_caption = torch.full((1,max_generate_length),50256)

        # In a loop, the model predicts one word at a time:
        for g in range(max_generate_length):
            # val_combined_embeds is passed to the PEFT model to get logits for the next word.
            phi_output_logits = peft_model(inputs_embeds=val_combined_embeds)['logits'] # 4, 69, 51200
            predicted_word_token_logits = phi_output_logits[:, -1, :].unsqueeze(1) # 4,1,51200
            # The word with the highest probability (logit) is selected using torch.argmax.
            predicted_word_token   = torch.argmax(predicted_word_token_logits, dim = -1) # 4,1
            # This predicted word is added to the predicted_caption tensor.
            predicted_caption[:,g] = predicted_word_token.view(1,-1).to('cpu')
            # The word is then embedded and concatenated to val_combined_embeds for the next prediction step.
            next_token_embeds      = phi_model.model.embed_tokens(predicted_word_token) # 4,1,2560
            val_combined_embeds    = torch.cat([val_combined_embeds, next_token_embeds], dim=1)

        # The predicted caption (which is in token format) is decoded into a human-readable string using the tokenizer.
        predicted_captions_decoded = tokenizer.batch_decode(predicted_caption,ignore_index = 50256)[0]

    print(f"Image: {val_image_url}")
    print(f"Question: {val_q}")
    print(f"Answer:   {val_a}")
    print(f"Model Predicted Ans: {predicted_captions_decoded}")

model_run_val(sample_val_data,max_generate_length=100)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Image: http://images.cocodataset.org/train2017/000000150195.jpg
Question: What are some possible protein sources in this plate of food? [QA]
Answer:   The main protein source in this plate of food is the fried sausage, which may be made from pork, beef, or chicken, depending on the type of sausage used.<|endoftext|>
Model Predicted Ans: .                                                                                                   


In [ ]:
# The optimizer used here is Adam, a popular choice for gradient-based optimization
phi_optimizer        = torch.optim.Adam(peft_model.parameters(), lr=1e-6)
projection_optimizer = torch.optim.Adam(projection.parameters(), lr=1e-5)
resnet_optimizer     = torch.optim.Adam(resblock.parameters(),   lr=1e-5)

step = 0
running_loss = 0.

# This method is called on the layers or model components to set them to training mode.
# This mode activates certain behaviors like dropout and batch normalization, which are typically used during training but disabled during evaluation or inference.
# projection.train(): Prepares the projection layer to update its weights.
# peft_model.train(): Sets the PEFT model in training mode.
# resblock.train(): Prepares the residual block for training.
projection.train()
peft_model.train()
resblock.train()

SimpleResBlock(
  (pre_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
  (proj): Sequential(
    (0): Linear(in_features=2560, out_features=2560, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=2560, out_features=2560, bias=True)
  )
)

In [ ]:
# The outer loop iterates through 3 training epochs
for epoch in tqdm(range(3), desc="Epochs"):
    # The inner loop iterates over batches from the train_dataloader
    for batch_idx, (images, questions, answers) in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch {epoch+1}", leave=False):
        # This checks if any of the batch components (images, questions, or answers) are empty or None. If they are, the batch is skipped.
        if (images is None or len(images) == 0) or (questions is None or len(questions) == 0) or (answers is None or len(answers) == 0):
            continue
        # Move data to device
        images = {'pixel_values': images.to(device)}
        questions = questions.to(device)
        answers = answers.to(device)

        # CLIP processing
        # clip_model processes the images through the CLIP model, producing the embeddings (output features).
        # The CLS token (classification token) is removed from the embeddings.
        clip_outputs = clip_model(**images)
        images_embeds = clip_outputs.last_hidden_state[:, 1:, :]  # remove cls token

        # Projection and processing
        # projection: The image embeddings from CLIP are projected to match the dimensionality of the Phi model.
        image_embeds = projection(images_embeds)
        # resblock: The embeddings are passed through a residual block to further refine them.
        image_embeds = resblock(image_embeds).to(torch.float16)

        # Token embedding
        # img_token_tensor: Creates a tensor representing a special image token.
        img_token_tensor = torch.tensor(IMAGE_TOKEN_ID).repeat(questions.size(0), 1).to(device)
        # img_token_embeds: Embeds the image token using the Phi model’s embedding layer.
        img_token_embeds = peft_model.model.model.embed_tokens(img_token_tensor)
        # questions_embed: Embeds the questions (text input) in the same way.
        questions_embed = peft_model.model.model.embed_tokens(questions)

        # Ensure dimensions match
        if image_embeds.shape[2] != img_token_embeds.shape[2]:
            padding_size = img_token_embeds.shape[2] - image_embeds.shape[2]
            image_embeds = F.pad(image_embeds, (0, padding_size))

        # Combine embeddings
        # Concatenates the image embeddings, image token embeddings, and question embeddings into one combined_embeds tensor.
        combined_embeds = torch.cat([image_embeds, img_token_embeds, questions_embed], dim=1)

        # Handle sequence length
        max_position_embeddings = peft_model.config.max_position_embeddings
        seq_length = combined_embeds.size(1)

        # The combined embeddings may exceed the model’s maximum allowed sequence length (max_position_embeddings and max_allowed_length), so the embeddings are truncated if necessary.
        if seq_length > max_position_embeddings:
            print(f"Warning: Sequence length ({seq_length}) exceeds maximum position embeddings ({max_position_embeddings}). Truncating.")
            combined_embeds = combined_embeds[:, :max_position_embeddings, :]
            seq_length = max_position_embeddings

        seq_length = combined_embeds.size(1)
        batch_size = combined_embeds.size(0)

        # Ensure seq_length is within a reasonable range
        max_allowed_length = 2048  # Adjust this based on your model's capabilities
        if seq_length > max_allowed_length:
            print(f"Warning: Sequence length ({seq_length}) exceeds maximum allowed length ({max_allowed_length}). Truncating.")
            combined_embeds = combined_embeds[:, :max_allowed_length, :]
            seq_length = max_allowed_length

        # Create position IDs with error handling
        # position_ids: A tensor representing the positional indices for the input sequence. This ensures the model can recognize the position of each token.
        try:
            position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
            position_ids = position_ids.unsqueeze(0).expand(batch_size, -1)
        except RuntimeError as e:
            position_ids = torch.arange(seq_length, dtype=torch.long)
            position_ids = position_ids.unsqueeze(0).expand(batch_size, -1).to(device)

        # Forward pass
        # phi_output: The combined embeddings are passed through the PEFT-tuned Phi model to produce the logits (predictions).
        # The output logits are trimmed to focus on the part that corresponds to the answers, ignoring the image embeddings and token embeddings.
        phi_output = peft_model(inputs_embeds=combined_embeds, position_ids=position_ids)
        phi_output_logits = phi_output.logits

        # Process output logits
        phi_output_logits = phi_output_logits[:, images_embeds.shape[1] + 1:, :]
        phi_output_logits = phi_output_logits.reshape(-1, peft_model.config.vocab_size)

        phi_optimizer.zero_grad()
        projection_optimizer.zero_grad()
        resnet_optimizer.zero_grad()

        # loss: The cross-entropy loss is calculated between the model’s predictions (phi_output_logits) and the ground truth answers.
        loss = F.cross_entropy(phi_output_logits, answers.contiguous().view(-1), ignore_index=50296, label_smoothing=0.1)

        # loss backprop
        #loss.backward(): Computes gradients for the loss.
        loss.backward()
        # Optimizers (phi_optimizer, projection_optimizer, resnet_optimizer) perform gradient descent updates for their respective parameters.
        phi_optimizer.step()
        projection_optimizer.step()
        resnet_optimizer.step()

        if step % model_log_step == 0:
            tqdm.write(f"Iteration {step}/{max_steps}, Loss: {loss.item()}")

        # Switch to eval mode and save
        if step % model_val_step == 0:
            projection.eval()
            peft_model.eval()
            resblock.eval()
            model_run_val(sample_val_data, max_generate_length)
            projection.train()
            peft_model.train()
            resblock.train()

        if step % model_save_step == 0:
            tqdm.write("Saving Checkpoint")
            torch.save(projection.state_dict(), os.path.join(os.getcwd(), "model_chkpt/finetunned_projection.pth"))
            torch.save(resblock.state_dict(), os.path.join(os.getcwd(), "model_chkpt/finetuned_resblock.pth"))
            peft_model.save_pretrained(os.path.join(os.getcwd(), "model_chkpt/lora_adaptor/"), save_adapter=True, save_config=True)

        if step >= max_steps:
            tqdm.write("Training finished.")
            break

        wandb.log({"step": step, "train_loss": loss.item()})
        step += 1

    tqdm.write(f"Epoch {epoch+1} completed.")

Epochs:   0%|          | 0/3 [00:00<?, ?it/s]
                                             t/s]
Epoch 1:   0%|          | 0/3125 [00:42<?, ?it/s]

Iteration 0/100000, Loss: 6.897550106048584


                                             
Epoch 1:   0%|          | 0/3125 [00:50<?, ?it/s]

Image: http://images.cocodataset.org/train2017/000000183398.jpg
Question: What are some of the main features of the kitchen? [QA]
Answer:   Some of the main features of the kitchen include an island bar, a kitchen sink, cupboards, a bowl of fruit, and a coffee maker.<|endoftext|>
Model Predicted Ans:  and a. and. and. and.

and. a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and a. and
Saving Checkpoint



Epoch 1:   3%|▎         | 99/3125 [06:39<1:02:26,  1.24s/it]
                                             59,  1.07s/it] 
Epoch 1:   3%|▎         | 100/3125 [07:13<53:59,  1.07s/it]

...

Iteration 7000/100000, Loss: 3.4088711738586426


                                                            
Epoch 3:  24%|██▍       | 750/3125 [48:58<43:51,  1.11s/it]

Image: http://images.cocodataset.org/train2017/000000491579.jpg
Question: What color is the toaster in the image? [QA]
Answer:   The toaster in the image is silver.<|endoftext|>
Model Predicted Ans: The toaster in the image is white.<|endoftext|>

## HARNESSING THE POWER OF TECHNOLOGY

Technology has become an integral part of our daily lives, from the smartphones we carry in our pockets to the smart homes we live in. It has revolutionized the way we communicate, work, and interact with the world around us. However, with great power comes great responsibility, and it is important to harness the power of technology in a way that benefits society as a whole.

Saving Checkpoint



Epochs:  67%|██████▋   | 2/3 [7:35:25<3:47:42, 13662.79s/it] 


UnidentifiedImageError: Caught UnidentifiedImageError in DataLoader worker process 6.
Original Traceback (most recent call last):
  File "/home/cudaq/.local/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 309, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/home/cudaq/.local/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/cudaq/.local/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/work/IG (5087744)/llavadataset.py", line 30, in __getitem__
    image_load = Image.open(requests.get(img_url,stream=True).raw)
  File "/usr/local/lib/python3.10/dist-packages/PIL/Image.py", line 3498, in open
    raise UnidentifiedImageError(msg)
PIL.UnidentifiedImageError: cannot identify image file <_io.BytesIO object 